In [2]:
import os
import requests
import json
import pandas as pd
import random
import time
import re
import sqlite3
import datetime

from dotenv import load_dotenv, find_dotenv
import warnings
 
# 忽略所有的 Warning 警告
warnings.filterwarnings('ignore')
 
# 读取本地/项目的环境变量。

# find_dotenv() 寻找并定位 .env 文件的路径
# load_dotenv() 读取该 .env 文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())
coze_api_key=os.environ["COZE_API_KEY"]
COZE_BOT_ID = os.environ["COZE_BOT_ID"]

In [17]:
def call_coze_api(query,con_id='123',user_id='zmx'):
    # 填充调用Coze API获得回复的代码，返回为字典
    response_data = send_request(coze_api_key, con_id, COZE_BOT_ID, user_id, query)
    response = parse_message_object(response_data)
    if response["plugin"] != "":
        ref_info = plugin_text_process(response["plugin"])
        response.update(ref_info)
    return response

def send_request(personal_access_token, con_id, bot_id, user_id, query):
    # 填充调用Coze API的具体代码，获得coze的回复，返回为json格式
    url = 'https://api.coze.cn/open_api/v2/chat'
    
    headers = {
        'Authorization': f'Bearer {personal_access_token}',
        'Content-Type': 'application/json',
        'Accept': '*/*',
        'Host': 'api.coze.cn',
        'Connection': 'keep-alive'
    }
    
    data = {
        'conversation_id': con_id,
        'bot_id': bot_id,
        'user': user_id,
        'query': query,
        'stream': False
    }
    
    response = requests.post(url, headers=headers, json=data)
    
    return response.json()

def parse_message_object(message_dict):
    # 解析coze API返回的结果，以字典的形式返回
    # 初始化变量以存储结果
    plugin_data = ""
    last_answer_content = ""

    # 获取messages列表
    messages = message_dict.get('messages', [])

    # 遍历messages列表
    for message in messages:
        # 提取最后一个answer的content值
        if message.get('type') == 'answer':
            last_answer_content = message.get('content', "")

        # 提取plugin数据
        if message.get('type') == 'verbose':
            content_str = message.get('content', "")
            try:
                # 尝试将字符串解析为JSON对象
                content_data = json.loads(content_str)
                # 检查msg_type是否为stream_plugin_finish
                if content_data.get('msg_type') == 'stream_plugin_finish':
                    plugin_data = json.dumps(content_data.get('data'), ensure_ascii=False)
            except (json.JSONDecodeError, TypeError):
                # 如果解析失败或data不是字典，我们忽略这个条目
                continue

    # 构建结果字典
    result = {
        'plugin': plugin_data.strip(),
        'answer': last_answer_content.strip()
    }
    
    return result

def extract_bracket_content(text):
    # 正则表达式模式，匹配{}及其中的内容，但不包括{}
    pattern = r'({.*?})'
    # 使用findall方法找到所有匹配项
    matches = re.findall(pattern, text)
    return matches

def plugin_text_process(ref_info):
    # 将Plugin输出转化为字典
    # key = ref_web_url，ref_web_name，ref_std，think_process，result
    text = ref_info.replace('\\n', '')
    text = text.replace('\\\\\\\\\\\\\\', '\\')
    text = text.replace('\\\\\\', '')
    text = text[2:-2]
    text = extract_bracket_content(text)[0]
    text=eval(text)
    return text

def get_bot_direct_reply(api_response):
    bot_answer = api_response.get('answer', "")
    bot_answer = json.loads(bot_answer)
    reflection = bot_answer.get('reflection', "")
    intermediate = bot_answer.get('intermediate', "")
    classificationId = bot_answer.get('classificationId', "")
    answer = bot_answer.get('answer', "")
    return answer, reflection, intermediate, classificationId

In [18]:
query = "The circuits are connected by a switching relay."
api_response = call_coze_api(query)
answer, reflection, intermediate, classificationId = get_bot_direct_reply(api_response)
print(f'问题：{query}')
# 输出answer内容
print(f'写作类别：{classificationId}')
# 输出answer内容
print(f'中间结果：{intermediate}')
# 输出answer内容
print(f'结果：{answer}')
# 输出answer内容
print(f'修改意见：{reflection}')

问题：The circuits are connected by a switching relay.
写作类别：2
中间结果：A switching relay connects the circuits.
结果：The switching relay connects the circuits.
修改意见：["Rule 3.6: Consider revising to 'A switching relay connects the circuits.'", "Rule 2.3: Add the article 'the' before 'circuits' to maintain consistency with the original text."]


In [19]:
type(reflection)

list

In [25]:
# 使用pandas读取CSV文件
df = pd.read_csv('./tests/translation.csv', encoding='utf-8')
query_list = df['query'].tolist()
#for i in range(56,len(query_list)):
for i in range(len(query_list)):
    query = query_list[i]
    api_response = call_coze_api(query)
    answer, reflection, intermediate, classificationId = get_bot_direct_reply(api_response)
    if reflection != []:
        # 使用列表推导式将列表中的每个元素转换为字符串，并用换行符连接
        reflection = '<br>'.join(str(item) for item in reflection)
        reflection = "规则识别：<br>"+reflection
    else:
        reflection = ""
    print(f'问题：{query}')
    # 输出answer内容
    print(f'结果：{answer}')
    # 输出进度
    print(f'进度：{i+1}/{len(query_list)}')
    # 结果内容写入到df中
    df.loc[i, 'answer'] = answer
    df.loc[i, 'reflection'] = reflection
    df.loc[i, 'intermediate'] = intermediate
    df.loc[i, 'classificationId'] = classificationId

# df结果另存为文件
df.to_csv('./tests/result_trans.csv', index=False, encoding='utf-8', mode='w')

问题：Change the colour of the display.
结果：{
  "answer": "To change the colour of the display, you may need to access the settings menu of the device and look for the display or display settings option. Within this section, there should be an option to adjust the colour settings. The specific steps might vary depending on the type and model of the device you are using."
}
进度：1/12
问题：Tag the circuit breaker 36L7.
结果：对不起，暂时无法明确您的需求中关于写作风格的具体设定，请您补充相关信息，以便为您准确处理。
进度：2/12
问题：The main gear leg is held by the side stay.
结果：对不起，作为一个AI，暂时无法为你提供帮助。
进度：3/12
问题：The circuits are connected by a switching relay. (Passive)
结果：对不起，暂时无法为你提供帮助。
进度：4/12
问题：These values are used by the computer to calculate the energy consumption.
(Passive)
结果：
进度：5/12
问题：The test can be continued by the operator. (Passive)
结果：对不起，出现了一个错误。
进度：6/12
问题：On the ground, the valve can be opened with the override handle. (Passive)
结果：Open the valve with the override handle on the ground. **Edited Translation:**Open the valve with t